In [4]:
from pysat.solvers import Glucose3
from pysat.formula import CNF

def australia_map_sat_solver():
    # Create a SAT solver instance
    solver = Glucose3()

    # Variables representing colors for each region
    colors = ['R', 'G', 'B']

    # Assign variable indices for each region and color
    variable_indices = {}
    for region in australia_map:
        for color in colors:
            variable = f"{region}_{color}"
            variable_indices[variable] = len(variable_indices) + 1

    # Create CNF formula
    cnf_formula = CNF()

    # Constraint 1: Each region must be assigned exactly one color
    for region in australia_map:
        region_variables = [variable_indices[f"{region}_{color}"] for color in colors]
        cnf_formula.append(region_variables)

        for i in range(len(region_variables)):
            for j in range(i + 1, len(region_variables)):
                cnf_formula.append([-region_variables[i], -region_variables[j]])

    # Constraint 2: Adjacent regions must have different colors
    for region, neighbors in australia_map.items():
        for neighbor in neighbors:
            for color in colors:
                cnf_formula.append([-variable_indices[f"{region}_{color}"], -variable_indices[f"{neighbor}_{color}"]])

    # Add CNF formula to the solver
    solver.append_formula(cnf_formula.clauses)

    # Solve the SAT problem
    if solver.solve():
        # Get the solution
        solution = solver.get_model()

        # Extract the colors assigned to each region
        region_colors = {}
        for region, index in variable_indices.items():
            if solution[index - 1] > 0:
                region_name, color = region.split('_')
                region_colors[region_name] = color

        # Print the solution
        for region, color in region_colors.items():
            print(f"{region} -> {color}")
    else:
        print("No solution found.")

# Australia map with regions and their adjacent regions
australia_map = {
    'WA': ['NT', 'SA'],
    'NT': ['WA', 'SA', 'Q'],
    'SA': ['WA', 'NT', 'Q', 'NSW', 'V'],
    'Q': ['NT', 'SA', 'NSW'],
    'NSW': ['Q', 'SA', 'V'],
    'V': ['SA', 'NSW']
}

# Solve the Australia map coloring problem using SAT solver
australia_map_sat_solver()


WA -> R
NT -> G
SA -> B
Q -> R
NSW -> G
V -> R


In [1]:
from ortools.sat.python import cp_model

def australia_map_cp_sat_solver():
    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Variables representing colors for each region
    colors = ['R', 'G', 'B']
    color_indices = {color: i for i, color in enumerate(colors)}

    # Assign variable indices for each region and color
    region_vars = {}
    for region in australia_map:
        region_vars[region] = model.NewIntVar(0, len(colors) - 1, region)

    # Constraint 1: Each region must be assigned exactly one color
    # This is implicitly handled by the fact that each region has a single variable assigned to a value in the range of colors

    # Constraint 2: Adjacent regions must have different colors
    for region, neighbors in australia_map.items():
        for neighbor in neighbors:
            model.Add(region_vars[region] != region_vars[neighbor])

    # Create the solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Check the result
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        # Extract the colors assigned to each region
        region_colors = {}
        for region in australia_map:
            color_index = solver.Value(region_vars[region])
            region_colors[region] = colors[color_index]

        # Print the solution
        for region, color in region_colors.items():
            print(f"{region} -> {color}")
    else:
        print("No solution found.")

# Australia map with regions and their adjacent regions
australia_map = {
    'WA': ['NT', 'SA'],
    'NT': ['WA', 'SA', 'Q'],
    'SA': ['WA', 'NT', 'Q', 'NSW', 'V'],
    'Q': ['NT', 'SA', 'NSW'],
    'NSW': ['Q', 'SA', 'V'],
    'V': ['SA', 'NSW']
}

# Solve the Australia map coloring problem using CP-SAT solver
australia_map_cp_sat_solver()


ModuleNotFoundError: No module named 'ortools.sat'